## Homework 3 for ECON-GA 3002, Section 12

Arnav Sood, N11193569, `asood@nyu.edu` 

**Overview:** Given a CSV of companies by exchange ticker, company name, and market capitalization, this notebook plots stock volatility against market capitalization. Volatility is given as daily percent change in stock price, obtained by querying the Yahoo Finance API.

**Inelegances**: 
    - Everything. Everything here is ugly, but it works, so I guess there's some beauty in that.
    
Credit is due to StackExchange, especially for things like list comprehensions.    

In [35]:
# First, since we're only allowed three import statements, we might as well use them.
import requests as rq
import re
import matplotlib.pyplot as plt

In [27]:
# Now, open up the CSV file, which we downloaded. We're going to use the corrected one for now
path = '/Users/arnavsood/quantecon/quantecon_nyu_2016/homework_assignments/hw_set3/company_list_corrected.csv'
f = open(path,'r')
print(f)

<_io.TextIOWrapper name='/Users/arnavsood/quantecon/quantecon_nyu_2016/homework_assignments/hw_set3/company_list_corrected.csv' mode='r' encoding='UTF-8'>


In [28]:
# Array the lines. We can close f since we're done with it.
lines = f.read().split("\n")
f.close()

# Set up lines as lists
data = [0]*(len(lines)-1)
for i in range(len(lines)-1):
    data[i] = lines[i].split(',')
    
# This gives us a slight anomaly for companies with ", Inc." in their names, but we'll handle that later. It's not important, since we don't use the names anyway.
    
# Remove entries with missing data (i.e., 'N/A' market cap).
data = [entry for entry in data if entry[2] != 'N/A']
for entry in data:
    if len(entry) == 4 and entry[3] == 'N/A':
        data.remove(entry)
data.remove(data[0])
    
# Set up data receptacles.
syms = ["foo"]*len(data)
caps = [0.00]*len(data)
vols = [0.00]*len(data)

# Grab the tickers.
ticks = re.compile('[A-Z]+')
for i in range(0,len(data)):
    syms[i] = ticks.findall(data[i][0])[0]
    
# Group them by 200.
tickGroups = [syms[n:n+200] for n in range(0, len(syms), 200)]
    
# Grab the volatilities.
base = 'http://finance.yahoo.com/d/quotes.csv?s='
tickstr = base
results = [0]*14

for i in range(len(tickGroups)):
    for j in range(0,len(tickGroups[i])):
        if j+1 != len(tickGroups[i]):
            tickstr += tickGroups[i][j]+"+"
        else:
            tickstr += tickGroups[i][j]+"&f=p2"
            results[i] = rq.get(tickstr)
            tickstr = base

In [29]:
# Inspect results.
print(results)

[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>]


In [30]:
# Parse through gathered data.
nums = re.compile('\+\d+\.\d+|\-\d+\.\d+')
for i in range(len(results)):
    y = results[i].text.split("\n")
    # A manual correction, because of a returned '' at some point.
    if i == 13:
        del y[71]
    for j in range(len(y)):
        vols[200*i + j] = nums.findall(y[j])
        
for i in range(len(vols)):
    vols[i] = float(vols[i][0])

In [31]:
# Inspect volatilities.
print(vols[1:10])

[0.47, 0.62, 0.0, -1.29, 0.36, -1.619, 1.7, -4.6, 0.0]


In [32]:
# Parse market caps. The ugly way.
for i in range(len(data)):
    if len(data[i]) == 3:
        cap = data[i][2]
        if cap[len(cap)-1] == 'M':
            caps[i] = float(cap[0:len(cap)-1])*1000000
        elif cap[len(cap)-1] == 'B':
            caps[i] = float(cap[0:len(cap)-1])*1000000000
    elif len(data[i]) == 4:
        cap = data[i][3]
        if cap[len(cap)-1] == 'M':
            caps[i] = float(cap[0:len(cap)-1])*1000000
        elif cap[len(cap)-1] == 'B':
            caps[i] = float(cap[0:len(cap)-1])*1000000000

In [36]:
# Plot.
absvols = [abs(entry) for entry in vols]
plt.scatter(absvols,caps)
plt.xlabel('Absolute Volatility, Daily % Change')
plt.ylabel('Market Cap, USD')
plt.show()

![title](figure_1.png)